# Iterando en elementos en contenedores separados

4.11

- Problema
         Necesita realizar la misma operación en muchos objetos, pero los objetos están contenidos en diferentes contenedores y le gustaría evitar bucles anidados sin perder la legibilidad de su código.


- solución
         El método itertools.chain () se puede utilizar para simplificar esta tarea. Toma una lista de iterables como entrada y devuelve un iterador que enmascara de manera efectiva el hecho de que realmente estás actuando en varios contenedores.
         
Para ilustrarlo, considere este ejemplo:

In [2]:
from itertools import chain

In [3]:
a = [1, 2, 3, 4]
b = ['x', 'y', 'z']
for x in chain(a, b):
    print(x)

1
2
3
4
x
y
z


Un uso común de chain () es en programas en los que le gustaría realizar ciertos
operaciones en todos los elementos a la vez, pero los elementos se agrupan en diferentes
conjuntos.   
Por ejemplo:

```python
    # Various working sets of items
    active_items = set()
    inactive_items = set()
    # Iterate over all items
    for item in chain(active_items, inactive_items):
        # Process item
```

Esta solución es mucho más elegante que usar dos bucles separados, como se muestra a continuación:

```python
        for elemento in active_items:
            # Elemento de proceso
            ...

        for elemento in inactive_items:
            # Elemento de proceso
            ...
```

itertools.chain () acepta uno o más iterables como argumentos. Entonces funciona creando
un iterador que consume y devuelve sucesivamente los elementos producidos por cada uno de
los iterables suministrados que proporcionó. Es una distinción sutil, pero chain() es más eficiente que primero combinar las secuencias e iterar.   
Por ejemplo:

```python
        # Ineficiente
        for x in a + b:
            ...
        # Mejor
        for x in chain(a, b):
            ...
```


En el primer caso, la operación a + b crea una secuencia completamente nueva y, además
requiere que a y b sean del mismo tipo. chain () no realiza tal operación, por lo que está lejos más eficiente con la memoria si las secuencias de entrada son grandes y se puede aplicar fácilmente cuando los iterables en cuestión son de diferentes tipos.

# Crear canalizaciones de procesamiento de datos

4.12

- Problema
        Desea procesar datos de forma iterativa en el estilo de una canalización de procesamiento de datos (similar a Tuberías Unix). Por ejemplo, tiene una gran cantidad de datos que deben procesarse, pero no cabe del todo en la memoria.


- Solución
        Las funciones de generador son una buena forma de implementar canalizaciones de procesamiento.   

Para ilustrar, suponga que tiene un directorio enorme de archivos de registro que desea procesar:

In [4]:
import os
import fnmatch
import gzip
import re
import bz2
import zipfile

In [5]:
arch = os.walk(os.getcwd())

In [6]:
next(arch)

('d:\\recetas-python\\Capitulo 4 (Iterables y Generadotres)',
 [],
 ['arc.txt',
  'cap4-1-7.ipynb',
  'cap4-11-15-walrus-match-case.ipynb',
  'cap4-8-10.ipynb',
  'test'])

In [7]:
def gen_find(busqueda, top=os.getcwd()):
    """Encuentra archivos en un árbol de directorios que coinciden con un patrón.
    
    Utiliza os.walk() para recorrer recursivamente un árbol de directorios y 
    fnmatch para aplicar filtros de patrón tipo shell. Devuelve las rutas 
    completas de los archivos coincidentes.
    
    Args:
        busqueda (str): Patrón de búsqueda tipo shell (ej: '*.txt', 'cap4*.*')
        top (str): Ruta del directorio raíz donde comenzar la búsqueda 
                    (por defecto es el directorio actual)
        
    Yields:
        str: Ruta completa de cada archivo que coincide con el patrón
        
    Example:
        >>> for archivo in gen_find('*.py', '/home/user'):
        ...     print(archivo)
        /home/user/script1.py
        /home/user/subdir/script2.py
    """
    for path, carpetas, archivos in os.walk(top):
        for nombre in fnmatch.filter(archivos, busqueda):
            yield os.path.join(path, nombre)


In [8]:
archivos = gen_find("cap4*.*",os.getcwd())
archivos

<generator object gen_find at 0x000001E517C4BA40>

In [10]:
archivos=list(archivos)

In [9]:
def gen_opener(filenames):
    """Abre una secuencia de archivos, detectando automáticamente el formato.
    
    Lee nombres de archivo y abre cada uno, detectando automáticamente si es 
    gzip (.gz), bzip2 (.bz2) o zip (.zip). El archivo se cierra inmediatamente 
    al pasar a la siguiente iteración, permitiendo procesar archivos muy grandes 
    sin cargar todo en memoria.
    
    Args:
        filenames (iterable): Secuencia de rutas de archivo (strings)
        
    Yields:
        file object: Objeto de archivo abierto en modo lectura de texto
        
    Example:
        >>> for archivo in gen_opener(['log.txt', 'data.gz', 'archive.zip']):
        ...     primera_linea = archivo.readline()
        
    Note:
        Los archivos se cierran automáticamente después de cada iteración.
        Para archivos .zip, se lee el primer archivo dentro del ZIP.
    """
    for filename in filenames:
        if filename.endswith('.gz'):
            f = gzip.open(filename, 'rt')
        elif filename.endswith('.bz2'):
            f = bz2.open(filename, 'rt')
        elif filename.endswith('.zip'):
            # Para archivos ZIP, abre el primer archivo dentro del ZIP
            zf = zipfile.ZipFile(filename, 'r')
            first_file = zf.namelist()[0]
            f = zf.open(first_file, 'r')
        else:
            f = open(filename, 'rt')
        yield f
        f.close()

## Nota: Diferencia entre modo `'r'` y `'rt'`

La diferencia entre `'r'` y `'rt'` en Python es **muy sutil pero importante**:

- **`'r'`**: Abre en modo lectura (por defecto es texto)
- **`'rt'`**: Abre explícitamente en modo lectura de **TEXTO** (`'t'` = text mode)

### ¿Por qué usamos `'rt'` en este código?

En `gen_opener()` se usa `'rt'` por **coherencia y claridad**, especialmente porque también abrimos archivos comprimidos:

```python
if filename.endswith('.gz'):
    f = gzip.open(filename, 'rt')      # ← 'rt' explícito
elif filename.endswith('.bz2'):
    f = bz2.open(filename, 'rt')       # ← 'rt' explícito
else:
    f = open(filename, 'rt')           # ← 'rt' explícito
```

### Diferencia práctica en Windows:

```python
# Modo 'r' (texto con conversión automática en Windows)
with open('archivo.txt', 'r') as f:
    contenido = f.read()
    # En Windows: \r\n se convierte automáticamente a \n

# Modo 'rt' (equivalente, pero explícito)
with open('archivo.txt', 'rt') as f:
    contenido = f.read()
    # Mismo comportamiento, pero más claro
```

### Respuesta: ¿Es necesario?

**No es estrictamente necesario**. `'r'` sería suficiente. Se usa `'rt'` para:

1. ✅ Ser **explícito** - dejar claro que es modo texto
2. ✅ **Coherencia** - mantener uniformidad con `gzip.open()` y `bz2.open()`
3. ✅ **Legibilidad** - mejorar la comprensión del código

**Conclusión**: Puedes cambiar `'rt'` por `'r'` sin problema, funcionará igual. La `'t'` es solo para ser explícito y mejorar la legibilidad.

### Soporte para archivos ZIP

La función ahora soporta archivos `.zip`. Cuando detecta un archivo ZIP:
- Abre el ZIP
- Lee el nombre del primer archivo dentro del ZIP
- Abre ese primer archivo para lectura

**Limitación actual:** Solo procesa el primer archivo dentro del ZIP.

**¿Qué pasa si el ZIP tiene múltiples archivos?**

Si necesitas procesar **todos los archivos** dentro del ZIP, tienes varias opciones:

#### **Opción 1: Iterar sobre todos los archivos del ZIP**
```python
def gen_opener_all_files(filenames):
    """Abre archivos, procesando TODOS los archivos dentro de ZIPs"""
    for filename in filenames:
        if filename.endswith('.zip'):
            zf = zipfile.ZipFile(filename, 'r')
            for inner_file in zf.namelist():
                if not inner_file.endswith('/'):  # Ignorar directorios
                    f = zf.open(inner_file, 'r')
                    yield f
                    f.close()
        else:
            # ... resto del código para .gz, .bz2, etc
```

#### **Opción 2: Especificar qué archivo leer del ZIP**
```python
def gen_opener_with_index(filenames, zip_index=0):
    """Especifica qué archivo leer del ZIP (por índice)"""
    for filename in filenames:
        if filename.endswith('.zip'):
            zf = zipfile.ZipFile(filename, 'r')
            files = zf.namelist()
            if len(files) > zip_index:
                f = zf.open(files[zip_index], 'r')
                yield f
                f.close()
```

**Ejemplo de uso:**
```python
# Opción 1: Procesar todos los archivos
for f in gen_opener_all_files(['datos.zip', 'respaldo.gz']):
    print(f.readline())

# Opción 2: Procesar segundo archivo del ZIP
for f in gen_opener_with_index(['datos.zip'], zip_index=1):
    print(f.readline())
```

In [10]:
# Versión alternativa 1: Procesar TODOS los archivos dentro del ZIP
def gen_opener_all_files(filenames):
    """Abre archivos, procesando TODOS los archivos dentro de ZIPs.
    
    Si encuentra un ZIP con múltiples archivos, abre cada uno secuencialmente.
    """
    for filename in filenames:
        if filename.endswith('.gz'):
            f = gzip.open(filename, 'rt')
            yield f
            f.close()
        elif filename.endswith('.bz2'):
            f = bz2.open(filename, 'rt')
            yield f
            f.close()
        elif filename.endswith('.zip'):
            # Procesa TODOS los archivos dentro del ZIP
            zf = zipfile.ZipFile(filename, 'r')
            for inner_file in zf.namelist():
                if not inner_file.endswith('/'):  # Ignorar directorios
                    f = zf.open(inner_file, 'r')
                    yield f
                    f.close()
        else:
            f = open(filename, 'rt')
            yield f
            f.close()


# Versión alternativa 2: Especificar qué archivo leer del ZIP
def gen_opener_with_index(filenames, zip_index=0):
    """Abre archivos, permitiendo especificar cuál archivo leer del ZIP.
    
    Args:
        filenames: Rutas de archivo
        zip_index: Índice del archivo a leer del ZIP (por defecto 0 = primero)
    """
    for filename in filenames:
        if filename.endswith('.gz'):
            f = gzip.open(filename, 'rt')
            yield f
            f.close()
        elif filename.endswith('.bz2'):
            f = bz2.open(filename, 'rt')
            yield f
            f.close()
        elif filename.endswith('.zip'):
            zf = zipfile.ZipFile(filename, 'r')
            files = [f for f in zf.namelist() if not f.endswith('/')]
            if len(files) > zip_index:
                f = zf.open(files[zip_index], 'r')
                yield f
                f.close()
        else:
            f = open(filename, 'rt')
            yield f
            f.close()

In [11]:
list(gen_opener(archivos))


[<_io.TextIOWrapper name='d:\\recetas-python\\Capitulo 4 (Iterables y Generadotres)\\cap4-1-7.ipynb' mode='rt' encoding='cp1252'>,
 <_io.TextIOWrapper name='d:\\recetas-python\\Capitulo 4 (Iterables y Generadotres)\\cap4-11-15-walrus-match-case.ipynb' mode='rt' encoding='cp1252'>,
 <_io.TextIOWrapper name='d:\\recetas-python\\Capitulo 4 (Iterables y Generadotres)\\cap4-8-10.ipynb' mode='rt' encoding='cp1252'>,
 <_io.TextIOWrapper name='d:\\recetas-python\\Capitulo 4 (Iterables y Generadotres)\\cap4.tar.gz' encoding='cp1252'>]

In [12]:
def gen_concatenate(iterators):
    """Encadena múltiples iteradores en una sola secuencia continua.
    
    Toma una secuencia de iteradores y emite todos sus elementos 
    consecutivamente, simplificando la iteración sobre múltiples fuentes 
    de datos sin necesidad de bucles anidados.
    
    Args:
        iterators (iterable): Secuencia de iteradores a concatenar
        
    Yields:
        any: Elementos de todos los iteradores, en orden
        
    Example:
        >>> it1 = iter([1, 2, 3])
        >>> it2 = iter([4, 5, 6])
        >>> list(gen_concatenate([it1, it2]))
        [1, 2, 3, 4, 5, 6]
    """
    for it in iterators:
        yield from it

In [13]:
def gen_grep(pattern, lines):
    """Filtra líneas que coinciden con una expresión regular.
    
    Busca un patrón de expresión regular en una secuencia de líneas 
    y emite solo las que contienen una coincidencia.
    
    Args:
        pattern (str): Expresión regular a buscar
        lines (iterable): Secuencia de líneas de texto
        
    Yields:
        str: Líneas que coinciden con el patrón
        
    Example:
        >>> lineas = ['python es genial', 'java también', 'python rules']
        >>> list(gen_grep('python', lineas))
        ['python es genial', 'python rules']
    """
    pat = re.compile(pattern)
    for line in lines:
        if pat.search(line):
            yield line

In [15]:
lognames = gen_find('access-log*', '/home/emi')
files = gen_opener(lognames)
lines = gen_concatenate(files)
pylines = gen_grep('(?i)python', lines)
for line in pylines:
    print(line)

Si desea ampliar aún más la canalización, incluso puede alimentar los datos en expresiones generadoras. Por ejemplo, esta versión encuentra el número de bytes transferidos y suma el total:

In [ ]:
ognames = gen_find('access-log*', '/home/emi')
files = gen_opener(lognames)
lines = gen_concatenate(files)
pylines = gen_grep('(?i)python', lines)
bytecolumn = (line.rsplit(None,1)[1] for line in pylines)
bytes_ = (int(x) for x in bytecolumn if x != '-')
print('Total', sum(bytes_))

El procesamiento de datos de manera canalizada funciona bien para una amplia variedad de otros problemas, incluido el análisis, la lectura de fuentes de datos en tiempo real, el sondeo periódico, etc. Para comprender el código, es importante comprender que la declaración de rendimiento actúa como una especie de productor de datos, mientras que un bucle for actúa como un consumidor de datos. Cuando los generadores se apilan, cada rendimiento alimenta un solo elemento de datos a la siguiente etapa de la tubería que lo consume con iteración. En el último ejemplo, la función sum () en realidad está impulsando todo el programa, sacando un elemento a la vez de la tubería de generadores.
Una característica interesante de este enfoque es que cada función del generador tiende a ser pequeña y autónoma. Como tales, son fáciles de escribir y mantener. En muchos casos, tienen un propósito tan general que pueden reutilizarse en otros contextos. El código resultante que pega
los componentes juntos también tienden a leerse como una receta simple que se entiende fácilmente.  

La eficiencia de memoria de este enfoque tampoco puede ser exagerada. El código que se muestra funcionaría incluso si se usara en un directorio masivo de archivos. De hecho, debido a la naturaleza iterativa del procesamiento, se utilizaría muy poca memoria.  

Hay un poco de extrema sutileza en la función gen_concatenate (). El propósito de esta función es concatenar secuencias de entrada juntas en una secuencia larga de líneas. La función itertools.chain () realiza una función similar, pero requiere que todos los iterables encadenados se especifiquen como argumentos. En el caso de esta receta en particular, hacer eso implicaría una declaración como lines = itertools.chain (* files), lo que haría que el generador gen_opener () se consumiera por completo. Dado que ese generador está produciendo una secuencia de archivos abiertos que se cierran inmediatamente en el siguiente paso de iteración, no se puede usar chain (). La solución mostrada evita este problema.  

También aparece en la función gen_concatenate () el uso de yield from para delegar a un subgenerador. El resultado de la declaración de él simplemente hace que gen_concatenate () emita todos los valores producidos por el generador.  

Por último, pero no menos importante, debe tenerse en cuenta que un enfoque canalizado no siempre funciona para todos los problemas de manejo de datos. A veces, solo necesita trabajar con todos los datos a la vez. Sin embargo, incluso en ese caso, el uso de canalizaciones de generador puede ser una forma de dividir lógicamente un problema en una especie de flujo de trabajo.  



# Aplanar una secuencia anidada

4.13

- Problema
        Tiene una secuencia anidada que desea aplanar en una única lista de valores.


- Solución
        Esto se resuelve fácilmente escribiendo una función generadora recursiva que implique un rendimiento de declaración. 
        
Por ejemplo

In [17]:
def flatten(items, ignore_types=(str, bytes)):
    for x in items:
        if isinstance(x, (list,tuple,set)): 
            yield from flatten(x)
        elif isinstance(x,(int,float,str)):
            yield x

In [18]:
items = [1, 2, [ 3, 4, [5, 6], 7 ], 8,{1,2,3},{1:"uno"}]

In [19]:
list(flatten(items))

[1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3]

In [25]:
nombres = ['Dave', 'Paula', ['Thomas', 'Lewis']]
for x in flatten(nombres):
    print(x)

Dave
Paula
Thomas
Lewis


In [20]:
def flatten2(items, ignore_types=(str, bytes)):
    from collections.abc import Iterable
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, ignore_types):
            yield from flatten(x)
        else:
            yield x

In [26]:
list(flatten2(items))

[1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 1]

In [27]:
list(flatten2(nombres))

['Dave', 'Paula', 'Thomas', 'Lewis']

En el código, isinstance (x, Iterable) simplemente comprueba si un elemento es iterable.
Si es así, yield from se utiliza para emitir todos sus valores como una especie de subrutina. El final resulto
es una única secuencia de salida sin anidamiento.

La declaración `yield from ` es un buen atajo para usar si alguna vez desea escribir generadores
que llaman a otros generadores como subrutinas. Si no lo usa, debe escribir un código que
utiliza un bucle for adicional. Por ejemplo:

```python
        def flatten(items, ignore_types=(str, bytes)):
            for x in items:
                if isinstance(x, Iterable) and 
                   not isinstance(x, ignore_types):
                    for i in flatten(x):
                        yield i
                else:
                    yield x
```

Aunque es solo un cambio menor, el rendimiento de la declaración simplemente se siente mejor y conduce a un código más limpio. Como se señaló, la verificación adicional de cadenas y bytes está ahí para evitar la expansión de esos tipos en caracteres individuales. Si hay otros tipos que no desea expandir, puede proporcionar un valor diferente para el argumento ignore_types. Finalmente, debe tenerse en cuenta que `yield` tiene un papel más importante en los programas avanzados que involucran corrutinas y concurrencia basada en generadores. Vea la receta 12.12 para
otro ejemplo.

# Iterando en orden clasificado sobre combinado clasificado Iterables

4.15

- Problema
        Tiene una colección de secuencias ordenadas y desea iterar sobre una secuencia ordenada de todos ellos se fusionaron.  
        
        

- Solución
        La función heapq.merge () hace exactamente lo que quieres.  

Por ejemplo:

In [28]:
import heapq

In [29]:
a = [1, 4, 7, 10]
b = [2, 5, 6, 11]

In [30]:
a + b

[1, 4, 7, 10, 2, 5, 6, 11]

In [31]:
for c in heapq.merge(a, b):
    print(c)

1
2
4
5
6
7
10
11


In [32]:
list(heapq.merge(a, b))

[1, 2, 4, 5, 6, 7, 10, 11]

La naturaleza iterativa de heapq.merge significa que nunca lee ninguna de los secuencias proporcionadas a la vez. Esto significa que puede usarlo en secuencias muy largas con muy poca sobrecarga.   
Por ejemplo, aquí hay un ejemplo de cómo fusionaría dos archivos:

import heapq
with open('archivo_1', 'rt') as file1, \
     open('archivo_2') 'rt' as file2, \
     open('archivo_final', 'wt') as outf:
    for line in heapq.merge(file1, file2):
        outf.write(line)

In [ ]:
import heapq
with open('archivo1', 'r') as file1, open('archivo2','r') as file2, \
     open('archivo_final', 'w') as outf:
    for line in heapq.merge(file1, file2):
        outf.write(line)

Es importante enfatizar que heapq.merge () requiere que todas las secuencias de entrada ya estén ordenadas. En particular, no lee primero todos los datos en un montón ni realiza ninguna clasificación preliminar. Tampoco realiza ningún tipo de validación de las entradas a comprobar
si cumplen con los requisitos de pedido. En su lugar, simplemente examina el conjunto de elementos del frente de cada secuencia de entrada y emite el más pequeño encontrado. Luego se lee un nuevo elemento de la secuencia elegida y el proceso se repite hasta que todas las secuencias de entrada
se han consumido por completo.

---

# 🔄 Modernización para Python 3.8+ 

Esta sección muestra cómo modernizar el código anterior aprovechando las características de Python 3.8+

## 1️⃣ Usar Pathlib en lugar de os.path (Python 3.4+)

**Versión antigua** (usando `os.path`):
```python
def gen_find(busqueda, top):
    for path, carpetas, archivos in os.walk(top):
        for nombre in fnmatch.filter(archivos, busqueda):
            yield os.path.join(path, nombre)
```

**Versión moderna** (usando `pathlib.Path`):
- ✅ Más legible y orientado a objetos
- ✅ Multiplataforma automático (Windows/Linux/Mac)
- ✅ Métodos integrados como `.glob()`, `.match()`, etc.

In [ ]:
from pathlib import Path

def gen_find_modern(pattern: str, top: str | Path):
    """
    Versión moderna usando pathlib.
    Encuentra todos los archivos que coinciden con el patrón.
    """
    top_path = Path(top) if isinstance(top, str) else top
    
    # glob() es más simple y directo que os.walk + fnmatch
    yield from top_path.rglob(pattern)  # rglob = recursive glob

In [ ]:
# Ejemplo de uso
archivos_modernos = list(gen_find_modern("cap4*.*", Path.cwd()))
print(f"Encontrados {len(archivos_modernos)} archivos:")
for archivo in archivos_modernos[:3]:  # Mostrar solo los primeros 3
    print(f"  - {archivo.name}")

## 2️⃣ Operador Walrus `:=` para simplificar código (Python 3.8+)

**Versión antigua**:
```python
def gen_opener(filenames):
    for filename in filenames:
        if filename.endswith('.gz'):
            f = gzip.open(filename, 'rt')
        elif filename.endswith('.bz2'):
            f = bz2.open(filename, 'rt')
        else:
            f = open(filename, 'rt')
        yield f
        f.close()
```

**Versión moderna con walrus**:
- ✅ Reduce repetición de código
- ✅ Más conciso y expresivo

In [ ]:
from pathlib import Path
from contextlib import contextmanager

@contextmanager
def open_file_smart(filepath: Path):
    """Context manager que abre archivos según su extensión."""
    if filepath.suffix == '.gz':
        f = gzip.open(filepath, 'rt')
    elif filepath.suffix == '.bz2':
        f = bz2.open(filepath, 'rt')
    else:
        f = open(filepath, 'rt')
    try:
        yield f
    finally:
        f.close()

def gen_opener_modern(filenames):
    """Versión moderna usando pathlib y context managers."""
    for filename in filenames:
        path = Path(filename) if isinstance(filename, str) else filename
        with open_file_smart(path) as f:
            yield f

## 3️⃣ Pattern Matching con match-case (Python 3.10+)

**Versión antigua** (flatten con if-elif):
```python
def flatten(items, ignore_types=(str, bytes)):
    for x in items:
        if isinstance(x, (list,tuple,set)): 
            yield from flatten(x)
        elif isinstance(x,(int,float,str)):
            yield x
```

**Versión moderna con match-case**:
- ✅ Más expresivo y legible
- ✅ Pattern matching estructural
- ✅ Detección de tipos más elegante

In [ ]:
from collections.abc import Iterable

def flatten_modern(items, ignore_types=(str, bytes)):
    """
    Versión moderna usando match-case (Python 3.10+)
    Aplana una secuencia anidada recursivamente.
    """
    for x in items:
        match x:
            # Si es iterable pero NO string/bytes
            case _ if isinstance(x, Iterable) and not isinstance(x, ignore_types):
                yield from flatten_modern(x, ignore_types)
            # Cualquier otro tipo (números, strings, etc.)
            case _:
                yield x

In [ ]:
# Prueba comparativa
items_test = [1, 2, [3, 4, [5, 6], 7], 8, {9, 10, 11}, 'texto']

print("Versión antigua:")
print(list(flatten(items_test)))

print("\nVersión moderna (match-case):")
print(list(flatten_modern(items_test)))

## 4️⃣ Type Hints Modernos (Python 3.9+)

**Cambios principales**:
- ✅ `list[int]` en lugar de `List[int]` (no necesita import de typing)
- ✅ `dict[str, int]` en lugar de `Dict[str, int]`
- ✅ `tuple[int, ...]` en lugar de `Tuple[int, ...]`
- ✅ `str | int` en lugar de `Union[str, int]` (Python 3.10+)
- ✅ `X | None` en lugar de `Optional[X]` (Python 3.10+)

In [ ]:
from collections.abc import Iterator, Iterable
from pathlib import Path

# Versión con type hints modernos (Python 3.10+)
def gen_grep_modern(
    pattern: str, 
    lines: Iterable[str]
) -> Iterator[str]:
    """
    Busca un patrón regex en líneas.
    
    Args:
        pattern: Expresión regular a buscar
        lines: Secuencia de líneas de texto
        
    Yields:
        Líneas que coinciden con el patrón
    """
    pat = re.compile(pattern)
    for line in lines:
        if pat.search(line):
            yield line

def gen_find_typed(
    pattern: str, 
    top: str | Path
) -> Iterator[Path]:
    """
    Encuentra archivos que coinciden con el patrón.
    
    Args:
        pattern: Patrón glob (ej: '*.txt')
        top: Directorio raíz (string o Path)
        
    Yields:
        Rutas de archivos encontrados
    """
    top_path = Path(top) if isinstance(top, str) else top
    yield from top_path.rglob(pattern)

## 5️⃣ Pipeline de procesamiento completo modernizado

Juntando todas las mejoras anteriores:

In [ ]:
# Pipeline completo modernizado (Python 3.10+)
from pathlib import Path
from collections.abc import Iterator
import re

# 1. Buscar archivos .ipynb en el directorio actual
notebooks = gen_find_typed("*.ipynb", Path.cwd())

# 2. Abrir archivos (simplificado - notebooks son texto plano)
def read_notebooks(paths: Iterator[Path]) -> Iterator[str]:
    """Lee líneas de múltiples archivos."""
    for path in paths:
        with path.open('r', encoding='utf-8') as f:
            yield from f

# 3. Filtrar líneas que contienen 'python'
lines = read_notebooks(notebooks)
python_lines = gen_grep_modern(r'(?i)python', lines)

# 4. Contar ocurrencias
print("Líneas con 'python' en notebooks:")
count = 0
for line in python_lines:
    if (count := count + 1) <= 5:  # Walrus operator! 
        print(f"{count}: {line.strip()[:60]}...")
    else:
        break

print(f"\n✅ Pipeline ejecutado con éxito (Python 3.10+)")

## 📊 Resumen de Modernizaciones

| Característica | Python < 3.8 | Python 3.8+ | Python 3.10+ |
|---------------|--------------|-------------|---------------|
| **Paths** | `os.path.join()` | `pathlib.Path` | `pathlib.Path` |
| **Type hints** | `List[int]` | `list[int]` | `str \| int` |
| **Asignación** | `x = expr; if x:` | `if (x := expr):` | `if (x := expr):` |
| **Pattern matching** | `if/elif/else` | `if/elif/else` | `match/case` |
| **Union types** | `Union[str, int]` | `Union[str, int]` | `str \| int` |
| **Optional** | `Optional[str]` | `Optional[str]` | `str \| None` |

### Beneficios de modernizar:

✅ **Código más conciso** - Menos boilerplate  
✅ **Mejor legibilidad** - Patrones más claros  
✅ **Type safety** - Mejor detección de errores con mypy/pylance  
✅ **Performance** - Algunas optimizaciones internas  
✅ **Mantenibilidad** - Aprovecha idiomas modernos de Python